Proyecto jiji

In [39]:
!pip install kagglehub


[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
import regex as re
import os
import pandas as pd
stemmer = SnowballStemmer('english')
import os
import pickle
import sys
import math
import dbm
import time
import heapq
import json

[nltk_data] Downloading package punkt to C:\Users\Ce
[nltk_data]     mar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
with open("stopwords-en.txt", encoding="latin1") as file:
   stoplist = [line.rstrip().lower() for line in file]
stoplist += ['?', '-', '.', ':', ',', '!', ';']

def preprocesamiento(texto, stemming=True):
  words = []
  texto = str(texto)
  texto = texto.lower()
  texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)
  # tokenizar
  words = nltk.word_tokenize(texto, language='english')
  # filtrar stopwords
  words = [word for word in words if word not in stoplist]
  # reducir palabras (stemming)
  if stemming:
      words = [stemmer.stem(word) for word in words]
  return words


In [42]:
# Funciones AUXILIARES para recuperar la información

def getNumberWithAtributo(dataset_head, atributo):
    if atributo in dataset_head.columns:
        return dataset_head.columns.get_loc(atributo)  # Obtiene la posición del atributo
    return -1  # Retorna -1 si no existe
 

def getResultados(result, path, disk_limit):
    res = []
    for chunk in pd.read_csv(path, chunksize=disk_limit):
        for doc, score in result:
            if doc in chunk.index:
                res.append((chunk.iloc[doc], score))
    return pd.DataFrame(res, columns=['doc_id', 'score'])  
def getResultadosDF(result, path, disk_limit):
    res = []
    for chunk in pd.read_csv(path, chunksize=disk_limit):
        for doc, score in result:
            row = chunk.iloc[doc].copy()
            row['score'] = score  
            res.append(row)
    return pd.DataFrame(res) 
def get_dfTex_Cols(path, row, columnas): #We don´t use it is only ofr simple an very fast testing
    dataset =  pd.read_csv(path)
    res = [dataset.iloc[row, i] for i in columnas]
    return ' '.join(res)  
#Funciones auxiliares para el mergeo:


In [43]:


class SPIMI:
    def __init__(self, index_dataset_path="path",bloques_dir="index_blocks" , columnas = ["track_name","track_artist","lyrics", "track_album_name"]):
        self.bloques_dir = bloques_dir  
        self.dataset_path = index_dataset_path  
        self.block_counter = 0      
        self.doc_ids = None  
        self.idf = {}
        self.length = {}
        self.disk_limit = 40000  
        
        #--Columnas:
        self.columnas = columnas 
        # ----Direciones de memoria para los files--
        self.dirIndex = self.bloques_dir + "_merge"
        
        self.term_dict_path = os.path.join(self.dirIndex, 'term_dict.pkl')
        self.postings_file_path = os.path.join(self.dirIndex, 'postings.bin')
        self.norms_file_path = os.path.join(self.dirIndex, 'document_norms.pkl')
        #Guardar los ids
        self.doc_count_path = os.path.join(self.dirIndex, 'doc_count.txt')  

        if not os.path.exists(self.dirIndex):
            os.makedirs(self.dirIndex)
        if not os.path.exists(self.bloques_dir):
            os.makedirs(self.bloques_dir)

        if os.path.exists(self.doc_count_path): # Guardamos el total de docs en ves de usar un set
            with open(self.doc_count_path, 'r') as f:
                self.doc_ids = int(f.read().strip())

        self.load_doc_ids()   

    def load_doc_ids(self):
        if os.path.exists(self.doc_count_path):
            try:
                with open(self.doc_count_path, 'r') as f:
                    self.doc_ids = int(f.read().strip())
                    print(f"Documentos cargados: {self.doc_ids}")
            except (ValueError, FileNotFoundError):
                self.doc_ids = 0  # Si el archivo está vacío o corrupto
        else:
            self.doc_ids = 0
 
       
        
                  

    def spimi_invert(self):
        print("Creando y ordenando cada bloque")
        dictionary = {}
        doc_ids = set()
        time_spimiInvert_start = time.time()

        #Get columna indeces: 
        with open(self.dataset_path, mode='r', encoding='utf-8') as file:
            primera_linea = file.readline().strip()
        #Get number columns
        columnas = primera_linea.split(',')
        columnas_numbers  = [i for i in range(len(columnas)) if columnas[i] in self.columnas]

        # Creamos los diccionarios a los bloques
        for chunk in pd.read_csv(self.dataset_path,chunksize= self.disk_limit): 
            for doc_id_, row in chunk.iterrows():
                self.doc_ids +=1 # Actualizamos la cantidad de id
                preFila = [str(row.iloc[i]) for i in columnas_numbers]
                texto = ' '.join(item for item in preFila)
                words = preprocesamiento(texto)
                for text in words:
                    doc_id = doc_id_
                    token = text
                    if token not in dictionary:
                        dictionary[token] = {}  

                    if doc_id not in dictionary[token]:
                        dictionary[token][doc_id] = 1  
                    else:
                        dictionary[token][doc_id] += 1  

                    dictionary_size = sys.getsizeof(dictionary)
                    if dictionary_size >= self.disk_limit:
                        self.write_block_to_disk(dictionary)
                        dictionary.clear()

        if dictionary:
            self.write_block_to_disk(dictionary)  
        
        with open(self.doc_count_path, 'w') as f:
            f.write(str(self.doc_ids))

        time_spimiInvert = time.time()- time_spimiInvert_start

        # MERGE
        print("Iniciando con el merge")
        time_Merge_start = time.time()
        self.mergeHeap()
        time_Merge = time.time() - time_Merge_start


        print("\nResumen de tiempos:")
        print(f"Tiempo para la creación Creación  y el ordenamiento de los bloques: {time_spimiInvert:.2f} segundos")
        print(f"Tiempo para la ejecución del merge: {time_Merge:.2f} segundos")
        print(f"Tiempo total para la creación y el mergeo: {time_Merge + time_spimiInvert:.2f} segundos")
    


    def write_block_to_disk(self, dictionary): # Guardo los ordenados
        # TO DO
        # Probar con guardar y luego ordenar con hilos
        sorted_terms = dict(sorted(dictionary.items())) 
        file_path = os.path.join(self.bloques_dir, f"block_{self.block_counter}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            for term, postings in sorted_terms.items():
                postings_str = json.dumps(postings)  
                f.write(f"{term}: {postings_str}\n")  
        self.block_counter += 1


    def load_block(self, filepath):
        with open(filepath, "rb") as f:
            return pickle.load(f)
    def load_term_dict(self):
        term_dict = {}
        with open(self.term_dict_path, 'rb') as f:
            while True:
                try:
                    term_entry = pickle.load(f)
                    term_dict.update(term_entry)
                except EOFError:
                    break
        return term_dict

    def show_Block(self, index):
        print("Imprimiendo contenido del bloque: ", index)
        file_path = os.path.join(self.bloques_dir, f'block_{index}.pkl')
        if not os.path.exists(file_path):
            print(f"El bloque {index} no existe en la ruta: {file_path}")
            return
        try:
            with open(file_path, 'rb') as f:
                block_content = pickle.load(f)
            print(block_content)
        except Exception as e:
            print(f"Error al abrir el bloque {index}: {e}")



    # Ya se comprobó el correcto merge 
    # Ahora los archivo son terminos postings y salto de linea, asi se getea linea por línea 
    def mergeHeap(self):
        lista_bloques = os.listdir(self.bloques_dir)
        priority_queue = []
        idx = 0  # Índice único para asegurar comparabilidad en el heap
    
        postings_file_path = os.path.join(self.dirIndex, 'postings.bin')
        term_dict_path = os.path.join(self.dirIndex, 'term_dict.pkl')
        norms_file_path = os.path.join(self.dirIndex, 'document_norms.pkl')
    
        open_files = {}
        doc_norms_temp = {}  # Diccionario temporal para acumular las normas de cada documento
    
        try:
            for bloque_path in lista_bloques:  # Referencias de bloques y pusheo de una línea (term, postings)
                bloque_full_path = os.path.join(self.bloques_dir, bloque_path)
                open_files[bloque_path] = open(bloque_full_path, "r", encoding="utf-8")
                term_postings = self.read_next_term(open_files[bloque_path])
                if term_postings:
                    term, postings = term_postings
                    heapq.heappush(priority_queue, (term, idx, postings, bloque_path))
                    idx += 1
    
            current_term = None
            current_postings = {}
            postings_file_position = 0
    
            with open(postings_file_path, 'wb') as postings_file, \
                 open(term_dict_path, 'ab') as term_dict_file:
    
                while priority_queue:
                    term, _, postings, bloque_path = heapq.heappop(priority_queue)
                    file = open_files[bloque_path]
    
                    if term == current_term:  # Caso en que el término es igual
                        for doc_id, tf in postings.items():
                            current_postings[doc_id] = current_postings.get(doc_id, 0) + tf
                    else:  # Nuevo término
                        if current_term is not None:
                            # Calcular IDF
                            idf = math.log10(self.doc_ids / (1 + len(current_postings)))
    
                            # Guardar postings (TF e IDF) en postings.bin
                            tf_idf_postings = {}
                            for doc_id, tf in current_postings.items():
                                tf_weighted = math.log10(1 + tf)
                                tfidf = tf_weighted * idf
                                tf_idf_postings[doc_id] = {"tf": tf_weighted, "idf": idf}
    
                                # Acumular el cuadrado del TF-IDF en las normas temporales
                                doc_norms_temp[doc_id] = doc_norms_temp.get(doc_id, 0) + tfidf ** 2
    
                            postings_data = pickle.dumps(tf_idf_postings)
                            postings_file.write(postings_data)
    
                            # Guardar término en term_dict.pkl
                            pickle.dump({current_term: (postings_file_position, len(postings_data))}, term_dict_file)
                            postings_file_position += len(postings_data)
    
                        # Actualizar current_term y current_postings
                        current_term = term
                        current_postings = postings.copy()
    
                    next_term_postings = self.read_next_term(file)
                    if next_term_postings:
                        next_term, next_postings = next_term_postings
                        heapq.heappush(priority_queue, (next_term, idx, next_postings, bloque_path))
                        idx += 1
    
                # Procesar el último término
                if current_term is not None:
                    idf = math.log10(self.doc_ids / (1 + len(current_postings)))
                    tf_idf_postings = {}
                    for doc_id, tf in current_postings.items():
                        tf_weighted = math.log10(1 + tf)
                        tfidf = tf_weighted * idf
                        tf_idf_postings[doc_id] = {"tf": tf_weighted, "idf": idf}
    
                        # Acumular el cuadrado del TF-IDF en las normas temporales
                        doc_norms_temp[doc_id] = doc_norms_temp.get(doc_id, 0) + tfidf ** 2
    
                    postings_data = pickle.dumps(tf_idf_postings)
                    postings_file.write(postings_data)
                    pickle.dump({current_term: (postings_file_position, len(postings_data))}, term_dict_file)
                    postings_file_position += len(postings_data)
    
        finally:
            # Cerrar todos los archivos abiertos
            for file in open_files.values():
                file.close()
    
        # Guardar las normas finales tomando la raíz cuadrada
        final_norms = {int(doc_id): math.sqrt(value) for doc_id, value in doc_norms_temp.items()}
        with open(norms_file_path, 'wb') as norms_file:
            pickle.dump(final_norms, norms_file)
    
        self.term_dict_path = term_dict_path
        self.postings_file_path = postings_file_path
        print(f"Índice fusionado guardado en {postings_file_path}")
        print(f"Diccionario de términos guardado en {term_dict_path}")
        print(f"Normas de documentos guardadas en {norms_file_path}")


            # Realiza merge de todos los bloques:
            # Manejaremos un archivo para ubicar los un indice general self.indexTerms donde se guarda la posición del termino
            # En el otro archivo se guardaran completito tanto la palabra como el posting list pero ya ordenado
    
    def read_next_term(self, file):
        line = file.readline()
        if not line:  # Si llegamos al final del archivo
            return None
        term, postings_str = line.strip().split(":", 1)
        postings = json.loads(postings_str)  # Convertir la lista de postings de JSON a un diccionario
        return term, postings



    #--Funciones para testing--
    def print_all_norms(self):
        if not os.path.exists(self.norms_file_path):
            print("El archivo de normas no existe. Asegúrate de haber ejecutado 'spimi_invert' correctamente.")
            return

        try:
            with open(self.norms_file_path, 'rb') as norms_file:
                norms = pickle.load(norms_file)
                if not norms:
                    print("El archivo de normas está vacío.")
                    return

                print("Normas de los documentos:")
                for doc_id, norm in sorted(norms.items()):
                    print(f"Documento ID: {doc_id}, Norma: {norm}")
        except Exception as e:
            print(f"Error al leer o procesar las normas: {e}")

    def show_terms_and_positions(self):
        term_dict = self.load_term_dict()
        print("Términos y sus posiciones en el archivo de postings:")

        # for term, (position, length) in term_dict.items():
        #     print(f"Término: '{term}', Posición: {position}, Longitud: {length}")

        terms = list(term_dict.keys())
        if terms == sorted(terms):
            print("----->Yeeeei, Los términos están ordenados alfabéticamente en el diccionario final.")
        else:
            print("Los términos NO están ordenados alfabéticamente en el diccionario final.")


    def show_terms_with_postings(self):
        # Cargar el diccionario de términos desde el archivo term_dict.pkl
        term_dict = self.load_term_dict()

        if not os.path.exists(self.postings_file_path):
            print("El archivo de postings no existe. Ejecuta 'mergeHeap' primero.")
            return

        print("Términos y sus listas de postings:")
        try:
            with open(self.postings_file_path, 'rb') as postings_file:
                for term, (position, length) in term_dict.items():
                    postings_file.seek(position)  
                    postings_data = postings_file.read(length) 
                    postings = pickle.loads(postings_data) 
                    print(f"Término: '{term}', Postings: {postings}")
        except Exception as e:
            print(f"Error al mostrar los términos y postings: {e}")

    def retrieval(self, query, k):
        #Cargamos docs: 
        if self.doc_ids is None:
            self.load_doc_ids()

        if self.doc_ids == 0:  
            raise ValueError("El índice no está construido o no contiene documentos. Ejecuta 'spimi_invert' antes de realizar consultas.")
        print("Se cargo en total: ", self.doc_ids, " documentos")

        # Preprocesar la consulta
        terms = preprocesamiento(query)

        # Calcular el TF-IDF de la consulta
        tf_query = {}
        for term in terms:
            if term in tf_query:
                tf_query[term] += 1
            else:
                tf_query[term] = 1

        tfidf_query = {}
        norm_query = 0

        term_dict = self.load_term_dict()

        for term, tf in tf_query.items():
            if term in term_dict:  # Solo considerar términos presentes en el índice
                position, length = term_dict[term]
                # Calcular IDF basado en postings
                with open(self.postings_file_path, 'rb') as postings_file:
                    postings_file.seek(position)
                    postings_data = postings_file.read(length)
                    postings = pickle.loads(postings_data)  # Cargar postings
                    idf = math.log10(self.doc_ids / (1 + len(postings)))  # IDF

                tfidf_query[term] = math.log10(1 + tf) * idf
                norm_query += (tfidf_query[term]) ** 2
        norm_query = math.sqrt(norm_query) 

        scores = [0] * self.doc_ids  

        with open(self.postings_file_path, 'rb') as postings_file, \
             open(self.norms_file_path, 'rb') as norms_file:

            for term, w_tq in tfidf_query.items():
                if term in term_dict:
                    position, length = term_dict[term]
                    postings_file.seek(position)
                    postings_data = postings_file.read(length)
                    postings = pickle.loads(postings_data)  # Diccionario {doc_id: {'tf': tf, 'idf': idf}}

                    for doc_id_str, values in postings.items():
                        doc_id = int(doc_id_str)  # Convertimos la clave a entero
                        tfidf_td = values['tf'] * values['idf']
                        scores[doc_id] += tfidf_td * w_tq 

            # Normalizar las puntuaciones por la norma de los documentos
            norms_file.seek(0)
            norms = pickle.load(norms_file)
            # Convertir las claves de normas a enteros
            # norms = {int(k): v for k, v in norms.items()}

            for doc_id in range(self.doc_ids):
                doc_norm = norms.get(doc_id, 0)
                # print(f"Doc ID: {doc_id}, Score before normalization: {scores[doc_id]}, Doc Norm: {doc_norm}, Query Norm: {norm_query}")
                # print(f"Tipo de doc_id en postings: {type(doc_id)}, Tipo de doc_id en normas: {type(next(iter(norms.keys())))}")
                if doc_norm != 0:
                    scores[doc_id] /= (doc_norm * norm_query)

        # Ordenar las puntuaciones en orden descendente y devolver los k más relevantes
        result = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        return result[:k]




path = "spotify_2000.csv"
# ---------Creation---------
s = SPIMI( path)  
s.spimi_invert() 

# ---------End Creation---------
print("Se procesaron en total:  ", s.doc_ids)


#Prubas de merge
# s.show_terms_and_positions()
# s.show_terms_with_postings()
    

Creando y ordenando cada bloque
Iniciando con el merge
Índice fusionado guardado en index_blocks_merge\postings.bin
Diccionario de términos guardado en index_blocks_merge\term_dict.pkl
Normas de documentos guardadas en index_blocks_merge\document_norms.pkl

Resumen de tiempos:
Tiempo para la creación Creación  y el ordenamiento de los bloques: 28.40 segundos
Tiempo para la ejecución del merge: 1.45 segundos
Tiempo total para la creación y el mergeo: 29.85 segundos
Se procesaron en total:   2000


In [46]:


# ----------Set Atributo--------
query = "this is my breath"
query = get_dfTex_Cols(path, 4, [1,2,3,6])
top_k = 20

result = s.retrieval(query, top_k)
respuesta = getResultados(result, s.dataset_path, s.disk_limit) # La respuesta es un array pd de [fila del dataframe, score]
#-----------End of query------------

# s.print_all_norms()
getResultadosDF(result, s.dataset_path, s.disk_limit)

# print( "Scores: ")
# print( respuestadf["score"])

Se cargo en total:  2000  documentos


,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language,score
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,1,0.0409,0.03700,0.000000,0.1380,0.2400,130.018,193160,en,1.000000
1571,0PWqC3xDoCWyqy1IW3RiKE,Knockin',Angel Grant,Kinda cute your look is real fly But what do y...,19,67BjkhyDC5KzhYRiKDVCcX,Album,1998-01-01,Neo Soul Music,78RRvktrPMSqAoCI21mNOe,...,0,0.0369,0.55700,0.112000,0.0678,0.9080,105.919,294893,en,0.168544
755,0fbXFYtHPpq5fVNRGjxZrC,Scrub The Ground (feat. DJ Funk),Chocolate Puma,I got that work I got that work I got that wor...,58,1wwwzZ0LYiLOct5BdvNpRb,Scrub The Ground (feat. DJ Funk),2015-10-16,EDM 2019,1T0ed6Mg0QIruHvcoWWIKy,...,0,0.0518,0.02630,0.812000,0.3380,0.6820,126.982,168937,en,0.137517
959,0HUeFoQFiMUPKxFApxyPwL,Euphoria,Destiny Rogers,"All these feelings in my mind, I can't get it ...",53,2FHuZw1dqYbMHovcBF0E8L,Euphoria,2019-12-05,New R&B‏‏​​ ‍,4I6rTSxqKl1LRvES2O9owQ,...,0,0.0617,0.17900,0.000020,0.2080,0.6460,159.989,196749,en,0.136876
1392,0nJW01T7XtvILxQgC5J7Wh,When I Was Your Man,Bruno Mars,Same bed but it feels just a little bit bigger...,82,58ufpQsJ1DS5kq4hhzQDiI,Unorthodox Jukebox,2012-12-07,Today's Hits 2000-Present,6a66cg3HcsjYkisYyQcov6,...,1,0.0434,0.93200,0.000000,0.0880,0.3870,72.795,213827,en,0.119040
1323,0MPegzGbqTvO2tw7BqYUU8,Go On Girl,Ne-Yo,I can't get it back But I don't want it back I...,2,48ben69fRtEbMTGqGYU8Qw,Because Of You,2007-01-01,Ultimate Throwbacks Collection,1dsaMvnC1hXPCNGC4aVtjj,...,1,0.1090,0.15500,0.000000,0.0945,0.5620,83.896,261427,en,0.111420
1749,0SDgPPSjRPdGD3hyrCTG2S,Don't Worry Bout Me - Alle Farben Remix,Zara Larsson,"Everything, everything's cool now I wanted you...",48,3oEW0zWluwima7PXrGi8aF,Don't Worry Bout Me (Remixes),2019-05-03,Dance Pop Tunes,4SdfG4cPG3skmTiQLozZGh,...,1,0.2640,0.04470,0.000005,0.1780,0.3550,124.938,171914,en,0.100738
1559,0puf9yIluy9W0vpMEUoAnN,Bang Bang,Jessie J,She got a body like an hourglass But I can giv...,78,2rDIivxBafNKpgRqlzIRSb,Sweet Talker (Deluxe Version),2014-10-14,10er Playlist,1kEczIkZH8IgaWT2BiApxZ,...,0,0.0909,0.26000,0.000000,0.3800,0.7490,150.035,199387,en,0.100399
179,05hXrqBkWkAdl4kG0iNksE,Feel Me (feat. Kanye West),Tyga,"Yeah, yeah, yeah, yeah, yeah Aye this shit got...",0,3iWD9oq8nfhu3Jta42rzfu,B*tch I'm the Sh*t 2,2017-07-21,RAP Gangsta,1Z1gW89x4MSBjkvVjGg7DQ,...,1,0.1190,0.00362,0.000000,0.9020,0.1750,142.162,199447,en,0.089845
977,0I3wlS5dlpw0hv5zhxN0pz,We Like To Party,RetroVision,"Feel like to body play me, feel like to body p...",58,49QK3K9hkhbU3fBdFezvlP,We Like To Party,2019-05-17,Electro House - by Spinnin' Records,4pVZ70y8vzzkn2GVwQbQw8,...,0,0.1010,0.00764,0.017500,0.2170,0.2410,126.046,172381,en,0.088652


In [45]:

# def verificar_termino(tf_file_dir, term):
#     with dbm.open(tf_file_dir, "r") as tf_db:  
#         try:
#             term_key = term.encode()
#             postings_list = pickle.loads(tf_db[term_key])
#             print(f"Postings list para '{term}': {postings_list}")
#         except KeyError:
#             print(f"El término '{term}' no se encuentra en el archivo DBM.")

# # verificar_termino(s.tf_file_dir, "6") 